In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt
/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [65]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier as xg
from lightgbm import LGBMClassifier as lg
from sklearn.ensemble import RandomForestRegressor,GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier

In [66]:
import subprocess
from nltk.corpus import wordnet

try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')
    
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /kaggle/working/corpora/wordnet.zip
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


replace /kaggle/working/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


True

In [67]:
data=pd.read_csv(r"/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [68]:
df=data[:1000]

In [69]:
%%time
le=preprocessing.LabelEncoder()
df_le=le.fit_transform(df.sentiment)

CPU times: user 866 µs, sys: 0 ns, total: 866 µs
Wall time: 885 µs


In [70]:
df['Binry']=df_le

/tmp/ipykernel_32/2372769580.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
df.head()

,review,sentiment,Binry
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [72]:
print(df.shape)
print(data.shape)

(1000, 3)
(50000, 2)


In [73]:
%%time
import string
punctuations = string.punctuation
import re
def preprocessing(text):
    var = re.sub("[^a-zA-Z0-9]", " ", text)
    var = re.sub(r'^b\s+', '', text)
    var=var.lower()
    var=var.split()
    
    punct_tag=re.compile(r'[^\w\s]')
    var=punct_tag.sub(r'',text)
    
    html_tag=re.compile(r'<.*?>')
    var=html_tag.sub(r'',text)
    
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    var=emoji_clean.sub(r'',text)
    
    url_clean= re.compile(r"https://\S+|www\.\S+")
    var=url_clean.sub(r'',text)
    
    lemmatizer=WordNetLemmatizer()
    var = [lemmatizer.lemmatize(word) for word in var if not word in set(stopwords.words('english'))and word not in punctuations ]
    var=' '.join(var)
    return var
def preprocess_text(text):
    
    var = re.sub("[^a-zA-Z0-9]", " ", text)
    var = re.sub(r'^b\s+', '', text)
    var=re.sub(r'<.*?>','',text)
    var=var.lower()
    var=var.split()
    lemmatizer=WordNetLemmatizer()
    var = [lemmatizer.lemmatize(word) for word in var if not word in set(stopwords.words('english'))and word not in punctuations ]
    var=' '.join(var)
    return var

CPU times: user 24 µs, sys: 0 ns, total: 24 µs
Wall time: 30.8 µs


In [74]:
%%time
df.review=df.review.apply(preprocess_text)

CPU times: user 31.1 s, sys: 3.92 s, total: 35 s
Wall time: 35 s


<timed exec>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [75]:
def tfidf(data):
    tfidfv = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True, max_features=150000)
    fit_data_tfidf=tfidfv.fit_transform(data)
    return fit_data_tfidf

In [76]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense,Flatten,Conv2D,Conv1D,GlobalMaxPooling1D,GlobalMaxPool1D
from keras.optimizers import Adam
import numpy as np  
import pandas as pd 
import keras.backend as k
from keras.utils import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional,GRU
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

In [77]:
df.head()

,review,sentiment,Binry
0,one reviewer mentioned watching 1 oz episode h...,positive,1
1,wonderful little production. filming technique...,positive,1
2,thought wonderful way spend time hot summer we...,positive,1
3,basically there's family little boy (jake) thi...,negative,0
4,"petter mattei's ""love time money"" visually stu...",positive,1


In [78]:
df.review[0]

"one reviewer mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scene violence, set right word go. trust me, show faint hearted timid. show pull punch regard drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focus mainly emerald city, experimental section prison cell glass front face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealing shady agreement never far away.i would say main appeal show due fact go show dare. forget pretty picture painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high level graphic violence. violence, injustice (crooked guard who'll sold nickel, inmate who'll kill order get away it, well ma

In [79]:
maxlen=1000
max_features=5000 
embed_size=300

X=df.review
Y=df.Binry

train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=0.2,random_state=42,shuffle=True)

val_x=test_x
#Tokenizing steps- must be remembered
tokenizer=Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_x))
train_x=tokenizer.texts_to_sequences(train_x)
val_x=tokenizer.texts_to_sequences(val_x)

#Pad the sequence- To allow same length for all vectorized words
train_x=pad_sequences(train_x,maxlen=maxlen)
val_x=pad_sequences(val_x,maxlen=maxlen)
val_y=test_y
print("Padded and Tokenized Training Sequence".format(),train_x.shape)
print("Target Values Shape".format(),train_y.shape)
print("Padded and Tokenized Training Sequence".format(),val_x.shape)
print("Target Values Shape".format(),val_y.shape)

Padded and Tokenized Training Sequence (800, 1000)
Target Values Shape (800,)
Padded and Tokenized Training Sequence (200, 1000)
Target Values Shape (200,)


In [80]:
model=Sequential()
model.add(Embedding(max_features,embed_size,input_length=maxlen))
model.add(LSTM(60))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()
plot_model(
    model,
    to_file="simple_model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)
model.fit(train_x,train_y,batch_size=128,epochs=3,verbose=2,validation_data=(val_x,val_y))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1000, 300)         1500000   
                                                                 
 lstm_2 (LSTM)               (None, 60)                86640     
                                                                 
 dense_4 (Dense)             (None, 16)                976       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,587,633
Trainable params: 1,587,633
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
7/7 - 16s - loss: 0.6917 - accuracy: 0.5512 - val_loss: 0.6872 - val_accuracy: 0.6200 - 16s/epoch - 2s/step
Epoch 2/3
7/7 - 12s - loss: 0.6622 - accuracy: 0.8938 - va

In [82]:
train_sample=X
tokenizer=Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_sample))
train_sample=tokenizer.texts_to_sequences(train_sample)
train_sample=pad_sequences(train_sample,maxlen=maxlen)
word_index = tokenizer.word_index

In [84]:
from tqdm import tqdm
embeddings_index = {}
f = open('/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400000it [00:36, 11028.37it/s]

Found 400000 word vectors.


In [85]:
embedding_matrix = np.zeros((len(word_index) + 1, 200))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 18190/18190 [00:00<00:00, 196253.65it/s]


In [ ]:
inp=Input(shape=(maxlen,))
z=Embedding(len(word_index) + 1,200,weights=[embedding_matrix],trainable=False)(inp)
z=Bidirectional(LSTM(60,return_sequences='True',dropout=0.3, recurrent_dropout=0.3))(z)
z=GlobalMaxPool1D()(z)
z=Dense(16,activation='relu')(z)
z=Dense(1,activation='sigmoid')(z)
model=Model(inputs=inp,outputs=z)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()
plot_model(
    model,
    to_file="glove_simple_model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

model.fit(train_x,train_y,batch_size=128,epochs=3,verbose=2,validation_data=(val_x,val_y))